In [ ]:
# import libraries
import numpy as np
import sys
import psi4
#from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
import scipy
from scipy.optimize import curve_fit
from scipy import interpolate
from matplotlib import pyplot as plt
import matplotlib
from scipy import constants
from numpy.polynomial import Polynomial
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

In [ ]:
import os
print(os.path.abspath("."))

In [ ]:

cavity_free_E_array = np.load("/Users/proden/Code/SCQED-PCQED/array_data/MgHplus/fci_cavity_free_array_MgH_631g.npy")
cavity_E_array  = np.load("/Users/proden/Code/SCQED-PCQED/array_data/MgHplus/fci_cavity_arrays_MgH_631g.npy")
r_data = np.load("/Users/proden/Code/SCQED-PCQED/array_data/MgHplus/fci_r_array_MgH_631G.npy")


cavity_free_E_array = np.load("/Users/proden/Code/SCQED-PCQED/array_data/MgHplus/cis_cavity_free_array_MgH_ccpvdz.npy")
cavity_E_array  = np.load("/Users/proden/Code/SCQED-PCQED/array_data/MgHplus/cis_cavity_arrays_MgH_ccpvdz.npy")
r_data = np.load("/Users/proden/Code/SCQED-PCQED/array_data/MgHplus/cis_r_array_MgH_ccpvdz.npy")



cavity_free_E_array = np.load("/Users/proden/Code/SCQED-PCQED/array_data/MgHplus/cis_cavity_free_array_MgH_ccpvtz.npy")
cavity_E_array  = np.load("/Users/proden/Code/SCQED-PCQED/array_data/MgHplus/cis_cavity_arrays_MgH_ccpvtz.npy")
r_data = np.load("/Users/proden/Code/SCQED-PCQED/array_data/MgHplus/cis_r_array_MgH_ccpvtz.npy")

##  Scan centered around ground state PES minimum

In [ ]:
print(r_data[33])

In [ ]:

plt.plot(r_data, cavity_free_E_array[:, :])
plt.legend()
plt.show()
plt.plot(r_data, cavity_E_array[:, :])
plt.legend()
plt.show()


fci_S0 = cavity_free_E_array[:, 0]
fci_S1 = cavity_free_E_array[:, 3]


print(cavity_free_E_array[:, 1].argmin())

plt.plot(r_data, fci_S0)
plt.plot(r_data,fci_S1)
plt.legend()
plt.show()

plt.plot(r_data, fci_S0 + 0.20)
plt.plot(r_data, cavity_E_array[:, :])
plt.legend()
plt.show()

In [ ]:
np.argmin(cavity_free_E_array[:, 2])
np.argmin(cavity_free_E_array[:, 1])


print(r_data[35])
print(cavity_E_array[35, 2] - cavity_E_array[35, 0])



In [ ]:

plt.plot(r_data, cavity_free_E_array[:,0] + 0.1693322251699385)
plt.plot(r_data, cavity_free_E_array[:,2])
plt.legend()
plt.show()

In [ ]:
"""
delta = 0.0005
for i in range(0,1):
    for j in range( 0, int(cavity_E_array_HH.shape[1]/2)):
        array1 = cavity_E_array_HH[:, j]
        array2 = cavity_E_array_HH[:, j+1]

        #find closest points
        idx =(np.abs(array1 - array2)).argmin()

        #assume they crossover if they get really close
        if np.abs(array1[idx]- array2[idx]) < delta:

            print(j)
            print(j+1)
                
            #copy one of the arrays
            array1_copy = np.array(array1, copy=True)

            print(array1)

            array1 = np.concatenate([array2[:idx],  array1[idx:]])
            array2 =np.concatenate([array1_copy[:idx] , array2[idx:]])

            print(array1)

            cavity_E_array_HH[:,j] = array1
            cavity_E_array_HH[:,j+1] = array2
    

    


"""

In [ ]:
def declutter_E_array(E_array, discontinuity_threshold=0.0001, energy_diff_threshold = 0.02,  num_to_declutter = 2):

    E_array = np.copy(E_array)
    new_E_array = np.zeros_like(E_array)

    for i in range(0,num_to_declutter):
        previous_intersection = 0
        for z in range(0,50):
            for j in range(i+1, E_array.shape[1]):
                array1 = E_array[:, i]
                array2 = E_array[:, j]

                array1_from_previous_intersection = array1[previous_intersection:]
                array2_from_previous_intersection = array2[previous_intersection:]


                #find closest points
                closest_indices =np.where(np.abs(array1[previous_intersection:] - array2[previous_intersection:]) < energy_diff_threshold)
                if np.shape(closest_indices)[1] != 0:
                    #print(i)
                    #print(closest_indices)
                    pass


                try:
                    dy_1 = np.gradient(array1_from_previous_intersection)
                    idx_1 = np.where(abs(np.diff(dy_1)) >  discontinuity_threshold)[0]+1

                    dy_2= np.gradient(array2_from_previous_intersection)
                    idx_2 = np.where(abs(np.diff(dy_2)) > discontinuity_threshold)[0]+1

                    if (len(idx_1)!= 0 and len(idx_2) != 0 ):

                        mask_idx1_idx2 = np.isin(idx_1, idx_2)
                        indices_idx1_in_idx2 = np.where(mask_idx1_idx2)[0]
                        indices_idx1_in_idx2 = idx_1[indices_idx1_in_idx2]
                        #print(indices_idx1_in_idx2)


                        starting_index=0
                        for elem_index in range(len(indices_idx1_in_idx2)-1):
                            if indices_idx1_in_idx2[elem_index]+1 == indices_idx1_in_idx2[elem_index+1]:
                                starting_index = elem_index+1
                        indices_idx1_in_idx2 = indices_idx1_in_idx2[starting_index:]
                        
                            
                        

                        if(len(indices_idx1_in_idx2) != 0 ):

                            mask_discontinuties_energydiff = np.isin(indices_idx1_in_idx2, closest_indices)
                            indices_discontinuties_in_energydiff = np.where(mask_discontinuties_energydiff)[0]
                            #print(indices_discontinuties_in_energydiff)

                            if len(indices_discontinuties_in_energydiff) != 0 :

                                idx = indices_idx1_in_idx2[indices_discontinuties_in_energydiff[0]]+ previous_intersection
                                #print(idx)

                                array1_copy = np.array(array1, copy=True)

                                array1 = np.concatenate([array1[:idx],  array2[idx:]])
                                array2 =np.concatenate([array2[:idx] , array1_copy[idx:]])


                                E_array[:,i] = array1
                                E_array[:,j] = array2


                                previous_intersection = idx
                except():
                    print(i)

        new_E_array[:,i ] = E_array[:,i]

    return new_E_array

In [ ]:
cavity_E_array_0_05  = declutter_E_array(cavity_E_array, 0.00001, 0.002,num_to_declutter=12)

In [ ]:
plt.plot(cavity_E_array_0_05)
plt.ylim(-199.9, -199.5)
plt.show()

## Scan centered around excited state PES minimum

## Permanent Dipole Moment Calculations

## Calculation of k
Fit ground state PES of H2 to a quintic polynomial